# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库Milvus

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

In [20]:
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
from langchain_milvus import Milvus, BM25BuiltInFunction
my_emb = OllamaEmbeddings(base_url='http://129.201.70.35:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")

connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        }

vectordb = Milvus(
    embedding_function=my_emb,
    collection_name="ZXVMAXS6",
    vector_field=["dense", "sparse"],  # dense 列 + sparse 列
    connection_args=connection_args,
    consistency_level="Strong",
    builtin_function=BM25BuiltInFunction()  # 启用 BM25
)


2025-09-24 12:30:20,655 [DEBUG][_create_connection]: Created new connection using: async-http://localhost:19530 (async_milvus_client.py:599)


In [21]:
results = vectordb.similarity_search(query="什么是vmax的上网日志系统？", k=2)
results

[Document(metadata={'page': 13, 'subject': '', 'format': 'PDF 1.4', 'moddate': '', 'creator': 'DITA Open Toolkit', 'creationdate': '2023-05-23T21:45:33+08:00', 'title': '目录', 'producer': 'Apache FOP Version 2.6', 'trapped': '', 'file_path': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'pk': 461017040453078129, 'creationDate': "D:20230523214533+08'00'", 'source': '../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（上网日志业务）.pdf', 'keywords': '', 'total_pages': 29, 'author': '', 'modDate': ''}, page_content='4\xa0功能本章包含如下主题：\uf06c上网日志保存\n10\n\uf06c上网日志查询\n10\n\uf06c上网日志批量导入查询\n11\n\uf06c日志管理\n11\n\uf06c账号管理\n11\n\uf06c角色管理\n12\n\uf06c资源监控\n12\n\uf06c告警管理\n12\n\uf06c省级网关对接\n12\n\uf06c拨测结果自动比对功能\n12\n\uf06cNAT日志入库功能\n13\n\uf06c北向接口\n13\n\uf06c上网日志历史查询\n13\n\uf06c云化上网日志XDR查询\n13\n以下介绍ZXVMAX-S的主要的功能。\n4.1\xa0上网日志保存\uf06c支持使用Gbase数据库或HDFS保存上网日志。\uf06c上网日志保存时间可配置，最短保存7天时间，最长可保存一年时间。\uf06c支持自动清理超过保存时间的上网日志。\n4.2\xa0上网日志查询可通过web界面指定查询条件，查询用户上网日志。支持的查询条件：\uf06c时间范围+公网IP

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [22]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://129.201.70.35:11434', model='qwen3:8B', temperature=0.1)

# my_llm.invoke("你好")

## 3. 构建检索问答链

prompts

In [23]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [24]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    metadata_filter={"source": "../data_base/knowledge_path/VMAX-S/ZXVMAX-S（V6.23）产品描述（5GC业务）.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


#### 创建一个基于模板的检索链： 混合检索版本

混合检索

Milvus自带混合检索功能，
数据写入向量数据库流程需要修改

参考： https://milvus.io/docs/zh/milvus_hybrid_search_retriever.md

修改147-156行： 

In [25]:
# 如果使用Milvus的混合检索
            # vectordb = Milvus.from_documents(
            # documents=batch_docs,
            # embedding=my_emb,
            # builtin_function=BM25BuiltInFunction(),
            # vector_field=["dense", "sparse"],
            # collection_name="ZXVMAXS5",
            # drop_old=False,
            # connection_args=connection_args,
            # consistency_level="Strong",
            # )       

In [26]:
from langchain_milvus import Milvus, BM25BuiltInFunction
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://129.201.70.35:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS6",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
        builtin_function=BM25BuiltInFunction(),
        vector_field=["dense", "sparse"]
    )

2025-09-24 12:30:51,323 [DEBUG][_create_connection]: Created new connection using: async-http://localhost:19530 (async_milvus_client.py:599)


定义检索器，从向量数据库检索与问题相关的文档片段

In [27]:
retriever = vectordb.as_retriever()

将检索到的多个文档合并为单一字符串，作为上下文的输入

In [28]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

​​输入处理​​：  
RunnablePassthrough()直接传递用户问题到question字段。  
retriever | format_docs：检索文档并格式化为字符串，作为context。  
​​提示生成​​：prompt将context和question填充到模板中。  
​​模型调用​​：llm生成回答。  
​​输出解析​​：StrOutputParser将模型输出转为字符串。  

In [29]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | QA_CHAIN_PROMPT 
    | my_llm
    | StrOutputParser()
)


In [30]:
query = "ZXVMAX-S上网日志有哪些功能？"
res = rag_chain.invoke(query)
res


'<think>\n好的，用户问的是ZXVMAX-S上网日志有哪些功能。首先，我需要仔细查看提供的上下文内容，看看有没有直接提到上网日志相关的信息。\n\n首先，在产品定位与特点部分，提到了ZXVMAX-S端到端系统有用户分析、网络分析、专题分析和评估优化四大功能。还提到支持XDR查询、实时保障、多维分析、会话跟踪等功能。这里可能和上网日志有关，但需要更具体的信息。\n\n接下来，在产品特点部分，分层架构、可靠性设计等，但没有直接提到上网日志。然后，在可靠性设计中的软件可靠性部分，提到了容错能力、故障检测处理，但依然没有上网日志的内容。\n\n再仔细看产品描述的数据业务部分，提到ZXVMAX-S支持XDR查询、实时保障、多维分析、会话跟踪。这些功能可能与上网日志相关，比如XDR查询可能涉及用户数据记录，会话跟踪可能涉及上网行为的监控。但上下文中没有明确提到“上网日志”这个具体功能名称。\n\n用户的问题是关于上网日志的功能，而提供的上下文没有直接提到上网日志的具体功能点。因此，根据指示，如果不知道答案，应该直接说不知道，不编造。所以，正确的回答应该是告知用户当前上下文没有相关信息，因此无法提供答案，并在最后加上谢谢。\n</think>\n\n根据提供的上下文内容，没有明确提到ZXVMAX-S上网日志的具体功能。因此，我无法提供相关答案。谢谢你的提问！'

#### 加入ReRanker进行精排

第一阶段检索结束后，我们需要对候选数据重新排名，以获得更好的结果。您可以根据自己的要求选择加权排名器（WeightedRanker）或重新 排名 器（RRFRanker）。

WeightedRanker/RRF：是一种多路召回融合算法，通过加权融合不同检索路径（如向量搜索、关键词搜索）的排名结果，提升召回多样性。但它仅基于排名位置计算分数，不深入分析语义相关性。

Cohere等 Rerank：是基于深度学习的交叉编码器（Cross-Encoder），直接计算查询与文档的语义匹配分数，能更精准地识别上下文关联性，尤其适合处理复杂语义或多义词问题。

所以可以进一步进行Reranker

构建RAG

In [31]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_community.llms import Ollama
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate


from langchain_milvus import Milvus, BM25BuiltInFunction
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://129.201.70.35:11434', model="dengcao/Qwen3-Embedding-0.6B:F16")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="ZXVMAXS6",  # Milvus 集合名称
        connection_args={
            "host": "129.201.70.35",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
        builtin_function=BM25BuiltInFunction(),
        vector_field=["dense", "sparse"]
    )

my_llm = Ollama(base_url='http://129.201.70.35:11434', model='qwen3:8B', temperature=0.1)


template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


# 初始化重排模型（显式指定设备）
rerank_model = HuggingFaceCrossEncoder(
    # model_name="/opt/workspace/models/Qwen/Qwen3-Reranker-0.6B",
    model_name="/workspace/models/BAAI/bge-reranker-base",
)

# 配置重排器（调整 top_n 和批处理大小）
base_compressor = CrossEncoderReranker(
    model=rerank_model,
    top_n=5,  # 根据业务需求调整
    # batch_size=8  # 加速批量查询
    
)

# 创建压缩检索器（结合 Milvus 和 Rerank）
compression_retriever = ContextualCompressionRetriever(
    base_compressor=base_compressor,
    base_retriever=vectordb.as_retriever(search_kwargs={"k": 15})  # 初步召回 15 条
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | QA_CHAIN_PROMPT 
    | my_llm
    | StrOutputParser()
)

query = "ZXVMAX-S上网日志有哪些功能？"
res = rag_chain.invoke(query)
res

2025-09-24 12:32:11,201 [DEBUG][_create_connection]: Created new connection using: async-http://localhost:19530 (async_milvus_client.py:599)


'<think>\n好的，用户问的是ZXVMAX-S上网日志有哪些功能。我需要先仔细看看提供的上下文内容，找到相关部分。\n\n首先，在4.1节提到上网日志保存，支持Gbase或HDFS保存，保存时间可配置，7天到一年，并且自动清理过期日志。这部分是关于日志存储的。\n\n然后4.2节是上网日志查询，通过web界面，支持多种查询条件，比如时间范围加公网IP、目的IP、MSISDN、IMSI、URL，还可以组合这些条件，或者加上端口进行更精确的查询。这里详细列出了查询条件和组合方式。\n\n接下来还有其他功能，比如4.3可能提到批量导入查询，但用户的问题可能只关注日志保存和查询。不过需要确认是否有其他相关部分。比如后面提到的NAT日志入库、北向接口、历史查询、XDR查询等，这些可能属于上网日志相关的功能，但需要看是否在上下文中明确提到。\n\n另外，产品定位和特点里提到支持XDR查询、实时保障、多维分析等功能，但不确定是否直接属于上网日志的功能。需要区分哪些是日志管理相关的，哪些是其他分析功能。\n\n用户的问题明确是关于上网日志的功能，所以应该集中在4.1和4.2节，以及可能的其他相关部分。比如4.3可能提到批量导入查询，但原文中4.3可能没有详细内容，因为用户提供的上下文里4.3可能被省略了。需要检查用户提供的上下文是否有其他部分。\n\n在用户提供的上下文中，4.1和4.2是明确的，而其他如日志管理、账号管理等可能属于不同的功能模块，但可能和日志相关。不过问题只问上网日志的功能，所以应该只回答保存和查询相关的内容，以及可能的其他日志相关功能如批量导入查询、历史查询、XDR查询等，如果上下文中有提到的话。\n\n例如，在4.3可能提到上网日志批量导入查询，但用户提供的上下文里可能没有详细展开，但根据用户给出的目录，4.3是上网日志批量导入查询，可能属于功能之一。同样，上网日志历史查询和云化XDR查询可能也是相关功能。\n\n因此，综合来看，ZXVMAX-S上网日志的功能包括保存、查询、批量导入查询、历史查询、XDR查询，以及可能的NAT日志入库和北向接口等。但需要根据上下文中的具体描述来确认。\n\n在用户提供的上下文中，4.1和4.2是明确的，而其他如4.3可能提到批量导入查询，4.4日志管理可能涉及日志的管理功能，但不确定是否属于上网日志的直接功能。需要仔细检